Нумерация точек скелета

![image info](../reports/figures/notebook/skeletal_numbering.png)

In [ ]:
import os
from collections import defaultdict
import numpy as np
from glob import glob
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

In [ ]:
import cv2

In [ ]:
from IPython.display import Image, display
import tempfile
import os.path as osp
import json

In [ ]:
DISTANCE_MAP_SIZE = (60, 18)
AXIS_NUMBER = 2
WINDOW_SIZE = 3
IMAGE_BORDER = 10
POINT_NUMBER = 17
# BONE_LIST = [[15, 13], [13, 11], [16, 14], [14, 12], [11, 12],
#              [5, 11], [6, 12], [5, 6], [5, 7], [6, 8], [7, 9],
#              [8, 10], [1, 2], [0, 1], [0, 2], [1, 3], [2, 4],
#              [3, 5], [4, 6]]
BONE_LIST = [[15, 13], [13, 11], [16, 14], [14, 12], [11, 12],
             [5, 11], [6, 12], [5, 6], [5, 7], [6, 8], [7, 9],
             [8, 10]]
CENTER_BONE = [11, 12]
COLOR_MAP = plt.get_cmap('YlGn')

In [ ]:
csv_save_directory = "/usr/data/datasets/kalman-data/personal_folder/cva/graduate-work/data/processed/2d/"

In [ ]:
skeleton_save_directory = "/usr/data/datasets/kalman-data/personal_folder/cva/graduate-work/data/processed/2d/skeleton"
distant_map_save_directory = "/usr/data/datasets/kalman-data/personal_folder/cva/graduate-work/data/processed/2d/dist_map"

In [ ]:
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

In [ ]:
def sort_by_num(path):
    name = path.split("/")[-1]
    num = int(name.split(".")[0])
    return num

In [ ]:
def norm_vals(x):
    x = x-np.min(x)
    x = x/np.max(x)
    return x

In [ ]:
def get_distant_map(data, left, right):
    image = None
    for i in range(left, right+1):
        if data[i] is None:
            continue
        points = data[i][:,:AXIS_NUMBER]
        center_point = (points[CENTER_BONE[0], :] + points[CENTER_BONE[1], :])/2;
        points-=center_point
        points = np.append(points, np.zeros((points.shape[0], 1)), axis=1)
        if image is None:
            image = np.atleast_3d(points)
        else:
            image = np.append(image, np.atleast_3d(points), axis=2)
    if image is None:
        return None
    image = np.swapaxes(image, 1, 2)
    for i in range(POINT_NUMBER):
        for j in range(AXIS_NUMBER):
            image[i,:,j] = norm_vals(image[i,:,j])
    resize_image = cv2.resize(image*255, (POINT_NUMBER, WINDOW_SIZE*2+1), interpolation = cv2.INTER_AREA)
    return(resize_image)

In [ ]:
def plot_im_skeleton(skeleton, ax, color):       
    x = skeleton[:, 0]
    y = skeleton[:, 1]
    for bone in BONE_LIST:
        ax.plot([x[bone[0]], x[bone[1]]], [y[bone[0]], y[bone[1]]], color = color[:3])

In [ ]:
save = None

In [ ]:
def get_img(data, left, right):
    fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(3,3))
    xmin, xmax = None, None
    ymin, ymax = None, None
    
    colors = COLOR_MAP(np.linspace(0.0, 1, right-left+1))
    
    for i in range(left, right+1):
        if data[i] is not None:
            if xmin is None:
                xmin, xmax = data[i][:, 0].min(), data[i][:, 0].max()
                ymin, ymax = data[i][:, 1].min(), data[i][:, 1].max()
            else:
                xmin, xmax = min(xmin, data[i][:, 0].min()), max(xmax, data[i][:, 0].max())
                ymin, ymax = min(ymin, data[i][:, 1].min()), max(ymax, data[i][:, 1].max())
            plot_im_skeleton(data[i], ax, colors[i-left])

    if xmin is None:
        plt.close()
        return None
    
    if xmax-xmin > ymax-ymin:
        delta = ((xmax-xmin) - (ymax-ymin))/2
        ymax += delta
        ymin -= delta
    else:
        delta = ((ymax-ymin) - (xmax-xmin))/2
        xmax += delta
        xmin -= delta
        
    xmin -= IMAGE_BORDER
    xmax += IMAGE_BORDER
    ymin -= IMAGE_BORDER
    ymax += IMAGE_BORDER
    
    plt.xlim([xmin, xmax])
    plt.ylim([ymin, ymax])
    plt.gca().invert_yaxis()
    ax.set_axis_off()
    plt.close()
    return fig

In [ ]:
datasets = [f.path for f in os.scandir(detection_result) if f.is_dir()]
datasets = sorted(datasets, key = sort_by_num)

In [ ]:
data_dict = {'dist_map_path':[], 'skeleton_img_path':[], 'motion':[]}

for pdataset in datasets:
    ndataset = pdataset.split("/")[-1]
    print(f"INFO: dataset {ndataset}")
    
    examples = [f.path for f in os.scandir(pdataset) if f.is_dir()]
    for pexample in tqdm(examples):
        nexample = int(pexample.split("/")[-1])
        persons_desc_path = glob(f"{pexample}/*.json")
        for path in persons_desc_path:
            person_id = path.split("/")[-1].split(".")[0]
            with open(path) as f:
                data = json.load(f)
            action = data["action"]
            poses = data["pose"]
            
            for k, v in poses.items():
                if v is not None:
                    poses[k] = np.array(v)

            poses = {int(k):v for k,v in poses.items()}
            left = nexample - WINDOW_SIZE
            right = nexample + WINDOW_SIZE
            skeleton_img = get_img(poses, left, right)
            distant_map = get_distant_map(poses, left, right)
            
            if skeleton_img is None or distant_map is None:
                continue
            
            path_skeleton_img = f"{skeleton_save_directory}/{ndataset}/{nexample}"
            path_distant_map = f"{distant_map_save_directory}/{ndataset}/{nexample}"
            
            if not os.path.exists(path_skeleton_img):
                os.makedirs(path_skeleton_img)
            if not os.path.exists(path_distant_map):
                os.makedirs(path_distant_map)
            
            skeleton_img.savefig(f"{path_skeleton_img}/{person_id}.png")
            cv2.imwrite(f"{path_distant_map}/{person_id}.png", distant_map)
            
            data_dict['dist_map_path'].append(f"{path_distant_map}/{person_id}.png")
            data_dict['skeleton_img_path'].append(f"{path_skeleton_img}/{person_id}.png")
            data_dict['motion'].append(action)
    data_pd = pd.DataFrame.from_dict(data_dict)
    data_pd.to_csv(os.path.join(csv_save_directory, 'all_data.csv'))